# Groups Report  
Creates csv files for each course containing each student's group memberships  
Documentation: https://canvasapi.readthedocs.io

In [3]:
from canvasapi import Canvas
import pandas as pd

org_url = "https://website.edu"
token = "your-token"
canvas = Canvas(org_url, token)
course_id = ['list','of','course','ids']

In [4]:
for i in course_id:                 # iterate over each course
    course_name = []
    student_name = []
    student_id = []
    course = canvas.get_course(i)
    
    # make df of students in course
    for student in course.get_users(enrollment_type=['student']):
        course_name.append(course.name)
        student_name.append(student.name)
        student_id.append(student.id)
        
    students_df = pd.DataFrame({'course_name': course_name, 'student_name': student_name, 'student_id': student_id})
    
    # make df for each groupset
    big_df = students_df.copy()

    for groupset in course.get_group_categories():    # iterate over each groupset
            
        student_name = []
        student_id = []
        group_name = []
        
        for group in groupset.get_groups():           # iterate over each group  
            for student in canvas.get_group(group).get_memberships():      # iterate over each student in current_group
                student_id.append(student.user_id)
                group_name.append(group.name)

        groups_df = pd.DataFrame({'student_id': student_id, groupset.name: group_name})
        big_df = big_df.merge(groups_df, how='left', on='student_id')
        
    big_df.fillna("-").to_csv(course.original_name[:7] + '_groups.csv', index=False)